In [1]:
#!pip install gensim
#!pip install python-Levenshtein

  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.2
    Uninstalling scipy-1.5.2:
      Successfully uninstalled scipy-1.5.2


In [5]:
pip install kagglehub

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: tqdm in c:\users\chima\anaconda3\lib\site-packages (from kagglehub) (4.67.1)



In [20]:
import gensim
import pandas as pd
import kagglehub
import os
import time

In [8]:
#Getting Amazon Product Review Data from the Kaggle



# Download the dataset
path = kagglehub.dataset_download("arhamrumi/amazon-product-reviews")

print("Path to dataset files:", path)

100%|███████████████████████████████████████████████████████████████████████████████| 115M/115M [00:01<00:00, 66.3MB/s]

Extracting model files...


Path to dataset files: C:\Users\chima\.cache\kagglehub\datasets\arhamrumi\amazon-product-reviews\versions\1


In [14]:
path_local = r'C:\Users\chima\.cache\kagglehub\datasets\arhamrumi\amazon-product-reviews\versions\1\Reviews.csv' 
df = pd.read_csv(path_local)
df.head(10)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [15]:
#Let's Preprocess the text present here 
Text = df.Text.apply(gensim.utils.simple_preprocess)

In [17]:
Text #This is the input to gensim model, a list of words. Sequence matters.

0         [have, bought, several, of, the, vitality, can...
1         [product, arrived, labeled, as, jumbo, salted,...
2         [this, is, confection, that, has, been, around...
3         [if, you, are, looking, for, the, secret, ingr...
4         [great, taffy, at, great, price, there, was, w...
                                ...                        
568449    [great, for, sesame, chicken, this, is, good, ...
568450    [disappointed, with, the, flavor, the, chocola...
568451    [these, stars, are, small, so, you, can, give,...
568452    [these, are, the, best, treats, for, training,...
568453    [am, very, satisfied, product, is, as, adverti...
Name: Text, Length: 568454, dtype: object

### Model Building

In [16]:
#Initialization

model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4,
)

In [21]:
t0 = time.time() #Start Time
model.build_vocab(Text, progress_per=1000)
t1 = time.time() #End Time
print(f"Time to build vocabulary: {t1 - t0:.2f} seconds")

Time to build vocabulary: 13.58 seconds


In [23]:
t0 = time.time() #Start Time
model.train(Text, total_examples=model.corpus_count, epochs=model.epochs)
t1 = time.time() #End Time
print(f"Time to build Model: {t1 - t0:.2f} seconds")

Time to build Model: 101.14 seconds


In [27]:
#Save this model rather than training it again and agian
model.save("./word2vec-amz-reviews-short.model") #~ 54MB model size. Wonder if we can optimize the size.

In [31]:
# Find words similar to a given word
similar_words = model.wv.most_similar("love", topn=20)  # Top 10 most similar words
for word, similarity in similar_words:
    print(f"Word: {word}, Similarity: {similarity}")

Word: adore, Similarity: 0.7567824125289917
Word: loves, Similarity: 0.6776376962661743
Word: enjoy, Similarity: 0.6320818066596985
Word: loved, Similarity: 0.6234462857246399
Word: enjoys, Similarity: 0.5778928995132446
Word: likes, Similarity: 0.5306829810142517
Word: loving, Similarity: 0.5296709537506104
Word: loooove, Similarity: 0.522752583026886
Word: fantastic, Similarity: 0.5199699401855469
Word: adores, Similarity: 0.5120050311088562
Word: yummy, Similarity: 0.5043520331382751
Word: fabulous, Similarity: 0.503262996673584
Word: enjoyed, Similarity: 0.49724164605140686
Word: awesome, Similarity: 0.4938930571079254
Word: like, Similarity: 0.49128180742263794
Word: delicious, Similarity: 0.48785728216171265
Word: terrific, Similarity: 0.4861878454685211
Word: wonderful, Similarity: 0.47923609614372253
Word: amazing, Similarity: 0.47516587376594543
Word: liked, Similarity: 0.4626755714416504


In [ ]:
t0 = time.time() #Start Time

#Find Words with highest Cosine Similarity
# Get all vocabulary words
words = list(model.wv.index_to_key)

# Find the most similar word pairs with cosine similarity > threshold
threshold = 0.8
similar_pairs = []
for i, word1 in enumerate(words):
    for word2 in words[i+1:]:  # Avoid duplicates (similarity(a, b) == similarity(b, a))
        similarity = model.wv.similarity(word1, word2)
        if similarity > threshold:
            similar_pairs.append((word1, word2, similarity))

# Sort by similarity
similar_pairs = sorted(similar_pairs, key=lambda x: -x[2])

# Print results
for word1, word2, similarity in similar_pairs[:20]:  # Show top 20
    print(f"Word1: {word1}, Word2: {word2}, Similarity: {similarity}")
    
t1 = time.time() #End Time
print(f"Time to build Model: {t1 - t0:.2f} seconds")